In [ ]:
# Notebook to compute total demand by TAZ for the automobile mode

import openmatrix as omx
import numpy as np
import pandas as pd
import geopandas as gp
import matplotlib.pyplot as plt
import bokeh
import xarray as xr
import hvplot.pandas
import hvplot.xarray

In [ ]:
%matplotlib notebook

In [ ]:
# Root directory for MoDX output for "base year" model results.
#
base_scenario_dir = r'G:/Regional_Modeling/1A_Archives/LRTP_2018/2016 Scen 00_08March2019_MoDXoutputs/'
#
# Root directory for MoDX output for "comparison scenario" model results.
# 
comparison_scenario_dir = r'G:/Regional_Modeling/1A_Archives/LRTP_2018/2040 NB Scen 01_MoDXoutputs/'

In [ ]:
# ===>>> USER INPUT REQUIRED: <<<===
#
# Supply path to root directory of scenario to use for the current run of this notebook:
# 
home_dir = comparison_scenario_dir
# 
# 2. Supply path to root of user's "sandbox" directory:
#
my_sandbox_dir = r'S:/my_modx_output_dir/'
#
# 3. Supply name of CSV output file for tabular results generated by this notebook:
#
csv_output_fn = 'taz_auto_report_comp.csv'

In [ ]:
taz_shapefile_base_dir = r'G:/Data_Resources/modx/canonical_TAZ_shapefile/'

In [ ]:
# trip_tables directory - this really "should" be a subdirectory of the base directory, but is isn't currently.
# The real McCoy - where things should go, and will eventually go
tt_dir = home_dir + 'out/'

In [ ]:
# trip tables OMX file (matrices)
tt_am = tt_dir + 'AfterSC_Final_AM_Tables.omx'
tt_md = tt_dir + 'AfterSC_Final_MD_Tables.omx'
tt_pm = tt_dir + 'AfterSC_Final_PM_Tables.omx'
tt_nt = tt_dir + 'AfterSC_Final_NT_Tables.omx'
trip_tables = { 'am' : omx.open_file(tt_am, 'r'),
                'md' : omx.open_file(tt_pm, 'r'),
                'pm' : omx.open_file(tt_pm,'r'),
                'nt' : omx.open_file(tt_nt, 'r') }

In [ ]:
num_tazes = trip_tables['am'].shape()[0]

In [ ]:
# Mapping from TAZ-ID to OMX index for the 4 periods (these *should* be the same)
taz_to_omxid_am = trip_tables['am'].mapping('ID')
taz_to_omxid_am = trip_tables['md'].mapping('ID')
taz_to_omxid_pm = trip_tables['pm'].mapping('ID')
taz_to_omxid_nt =  trip_tables['nt'].mapping('ID')

In [ ]:
# We'll assume that the mapping from TAZ ID to OMX ID doesn't vary by time period.
# We'll use the AM mapping as _the_ mapping for all time periods, pending confirmation.
# 
# TBD: Insert "sanity check" that the 4 mappings on "ID" are identical.
#
taz_to_omxid = taz_to_omxid_am

In [ ]:
# Function: load_tts_for_mode_list_as_np_arrays
#
# Summary: Load all trip tables for all time periods for a specified list of modes as NumPy arrays,
# and return a two-level dictionary (i.e., by time period and by mode) of the results.
#
def load_tts_for_mode_list_as_np_arrays(tts, mode_list):
    all_periods_list = all_periods_list = ['am', 'md', 'pm', 'nt']
    retval = {'am' : None, 'md' : None, 'pm' : None, 'nt' : None }
    for period in all_periods_list:
        retval[period] = {}
        for mode in mode_list:
            temp = tts[period][mode]
            retval[period][mode] = np.array(temp)
        # end_for
    # end_for
    return retval
# end_def load_tts_for_mode_list_as_np_arrays()

In [ ]:
temp = load_tts_for_mode_list_as_np_arrays(trip_tables, ['SOV', 'HOV'])

In [ ]:
# Compute total SOV, HOV, and total auto demand from each TAZ
sov_temp = temp['am']['SOV'] + temp['md']['SOV'] + temp['pm']['SOV'] + temp['nt']['SOV']
sov_demand = sov_temp.sum(axis=1)
hov_temp = temp['am']['HOV'] + temp['md']['HOV'] + temp['pm']['HOV'] + temp['nt']['HOV']
hov_demand = hov_temp.sum(axis=1)
total_auto_demand = sov_demand + hov_demand

In [ ]:
# Build dataframes of SOV, HOV, and total auto demand from each TAZ
# Set each data frame's index to the omxid of each row, i.e., its index
sov_df = pd.DataFrame(sov_demand, columns=['sov'])
sov_df['omxid'] = sov_df.index
sov_df.set_index('omxid')
hov_df = pd.DataFrame(hov_demand, columns=['hov'])
hov_df['omxid'] = hov_df.index
hov_df.set_index('omxid')
auto_df = pd.DataFrame(total_auto_demand, columns=['auto_total'])
auto_df['omxid'] = auto_df.index
auto_df.set_index('omxid')

In [ ]:
# Join the 3 dataframes into a single data frame
temp_df = pd.merge(left=auto_df, right=sov_df, on="omxid")
total_auto_trips_df = pd.merge(temp_df, hov_df, on="omxid")

In [ ]:
# Load the candidate canonical TAZ shapefile as a geopands dataframe.
# N.B. Use shapefile in WGS84 SRS.
#
taz_shapefile = taz_shapefile_base_dir + 'candidate_CTPS_TAZ_STATEWIDE_2019_wgs84.shp'
taz_gdf = gp.read_file(taz_shapefile)
taz_gdf.set_index("id")

In [ ]:
# Add a 'omxid' column to the TAZ geodataframe, in prep for joining with the total trips dataframes.
# ==> This also can be done earlier.
taz_gdf['omxid'] = taz_gdf.apply(lambda row: taz_to_omxid[row.id], axis=1)

In [ ]:
# Join the shapefile geodataframe to the total trips dataframe on 'omxid'
joined_df = taz_gdf.join(total_auto_trips_df.set_index('omxid'), on='omxid')

In [ ]:
# Export the useful columns of data in the 'joined_df' dataframe as a CSV file
fq_output_fn = my_sandbox_dir + csv_output_fn
joined_df.to_csv(fq_output_fn, sep=',', columns=['id', 'town', 'state', 'auto_total', 'sov', 'hov'])

In [ ]:
# Make a static map of total auto trips by origin TAZ
joined_df.plot("auto_total", figsize=(10.0,8.0), cmap='plasma', legend=True)
plt.title('Total Auto Trips by Origin TAZ')
plt.show()

In [ ]:
# Make an interactive map of the above
joined_df.hvplot(c='auto_total', 
                 geo=True, 
                 hover_cols=['id', 'town', 'auto_total', 'sov', 'hov'], 
                 clabel='Total Auto Trips', 
                 cmap='plasma',
                 frame_height=500).opts(title='Total Auto Trips by Origin TAZ')